# Exercises - Pricing a Callable Bond


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


# 1. Black's Formula for Bond Options


## 1.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `annual` frequency
* annualized coupon rate of `cpn=6%`.

Use the bond-pricing formula along with the discount rates in the data file to price this bond.


In [2]:
import numpy as np
import pandas as pd

curve = pd.read_excel('discount_curve_2025-02-13.xlsx', sheet_name='discount curve')
curve[['ttm','discount']] = curve[['ttm','discount']].apply(pd.to_numeric, errors='coerce')
curve = curve.dropna(subset=['ttm','discount']).sort_values('ttm')

times = np.array([1.0, 2.0, 3.0])
N = 100.0
cpn = 0.06
cashflows = np.array([N * cpn, N * cpn, N * (1 + cpn)])
df = np.interp(times, curve['ttm'].to_numpy(), curve['discount'].to_numpy())
pv = cashflows * df

out = pd.DataFrame({'t': times, 'cashflow': cashflows, 'discount': df, 'pv': pv})
price = float(pv.sum())

display(out)
print('Bond price:', price)

,t,cashflow,discount,pv
0,1.0,6.0,0.958451,5.750705
1,2.0,6.0,0.920515,5.523091
2,3.0,106.0,0.884084,93.712910


Bond price: 104.98670645677134


## 1.2.

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`
* vol of `2.68%`
* forward price of `103.31.`

What is the value of the issuer's call option?


In [3]:
import math
import numpy as np
import pandas as pd

curve = pd.read_excel('discount_curve_2025-02-13.xlsx', sheet_name='discount curve')
curve[['ttm','discount']] = curve[['ttm','discount']].apply(pd.to_numeric, errors='coerce')
curve = curve.dropna(subset=['ttm','discount']).sort_values('ttm')

T = 1.5
F = 103.31
K = 100.0
sig = 0.0268
Z = float(np.interp(T, curve['ttm'].to_numpy(), curve['discount'].to_numpy()))

N = lambda x: 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))
d1 = (math.log(F / K) + 0.5 * sig * sig * T) / (sig * math.sqrt(T))
d2 = d1 - sig * math.sqrt(T)
C = Z * (F * N(d1) - K * N(d2))

print('Z(0,T)=', Z)
print('Issuer call value=', C)

Z(0,T)= 0.9392282128072968
Issuer call value= 3.373836732035071


## 1.3.

What is the price of the callable bond? 

The **callable** bond is the bond issued with an embedded call option (long the issuer.) Thus, it is the value of the vanilla bond minus the value of the call option.


In [4]:
callable_price = float(price) - float(C)
print('Vanilla bond price=', float(price))
print('Issuer call value=', float(C))
print('Callable bond price=', callable_price)

Vanilla bond price= 104.98670645677134
Issuer call value= 3.373836732035071
Callable bond price= 101.61286972473627


## 1.4.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?


We prefer Black for this problem because the option is naturally quoted and modeled on a bond forward price, and discounting should use the full discount curve. Black matches these inputs directly, while Black-Scholes is a spot-price framework that typically relies on a simplified constant-rate discounting assumption.

## 1.5

Redo 1.2. Suppose the market prices the call option at `3.50`.

Solve for the implied volatility.


In [6]:
curve = pd.read_excel('discount_curve_2025-02-13.xlsx', sheet_name='discount curve')
curve[['ttm','discount']] = curve[['ttm','discount']].apply(pd.to_numeric, errors='coerce')
curve = curve.dropna(subset=['ttm','discount']).sort_values('ttm')

T = 1.5
F = 103.31
K = 100.0
Z = float(np.interp(T, curve['ttm'].to_numpy(), curve['discount'].to_numpy()))
target = 3.50

N = lambda x: 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def black_call(sig: float) -> float:
    if sig <= 0:
        return Z * max(F - K, 0.0)
    srt = sig * math.sqrt(T)
    d1 = (math.log(F / K) + 0.5 * sig * sig * T) / srt
    d2 = d1 - srt
    return Z * (F * N(d1) - K * N(d2))

lo, hi = 1e-10, 0.5
while black_call(hi) < target and hi < 10.0:
    hi *= 2.0

for _ in range(120):
    mid = 0.5 * (lo + hi)
    if black_call(mid) < target:
        lo = mid
    else:
        hi = mid

sigma_imp = 0.5 * (lo + hi)
print('Z(0,T)=', Z)
print('Implied vol=', sigma_imp)
print('Model price=', black_call(sigma_imp))

Z(0,T)= 0.9392282128072968
Implied vol= 0.03094057783932508
Model price= 3.5000000000000058
